In [1]:
import cramer_shoup as cs
from web3 import Web3
from solcx import compile_files
from py_ecc import bn128
import secrets

In [13]:
private_key, public_key = cs.generate_keypair()
message = cs.get_random_generator() # TODO create a mapping between the message and the curve
message

(15754522824298298678984299068687499050270661613918382033006769500817477230367,
 17009251104546859774124359615545239626005643362554707521151395815650216148847)

In [19]:
chiper_text = cs.encrypt(public_key, message)
chiper_text

((7871307926219681742487650246470518659441072948152725651824610276909180789788,
  9476438050457643673799321977840582521930143872021008928068318312342161208614),
 (5253350212122926731145278693632563171991061264376397068520408919911828018614,
  14837305404918556641708472945347301957784539799811047484444329220267761239472),
 (15401757562686764749818126492976018073145854999039664850292627653501494626819,
  5856512345977014747143158677378472758348160848472688865538423528949603261885),
 (3907119855844795814535011561213244514277085447620037117548260804903678815501,
  14688904973713871556760779308848336367744387485761805180249590165278996412073),
 1254312500573931302050004706462938473998528966027706767374135875781676719502)

In [29]:
w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
compiled_sol = compile_files(["cramer_shoup.sol"], output_values=['abi', 'bin'])['cramer_shoup.sol:CramerShoup']
CramerShoup = w3.eth.contract(address='0xD7ACd2a9FD159E69Bb102A1ca21C9a3e3A5F771B', abi=compiled_sol['abi'])

Traceback (most recent call last):
  File "/opt/homebrew/bin/solc", line 8, in <module>
    sys.exit(solc())
  File "/opt/homebrew/lib/python3.10/site-packages/solc_select/__main__.py", line 80, in solc
    res = current_version()
  File "/opt/homebrew/lib/python3.10/site-packages/solc_select/solc_select.py", line 60, in current_version
    raise argparse.ArgumentTypeError(
argparse.ArgumentTypeError: No solc version set. Run `solc-select use VERSION` or set SOLC_VERSION environment variable.


CalledProcessError: Command '['/opt/homebrew/bin/solc', '--version']' returned non-zero exit status 1.

In [5]:
tx_hash = CramerShoup.functions.setPublicKey(
    _pk = {
        "g1" : cs.point_to_bytes(public_key['g1']),
        "g2" : cs.point_to_bytes(public_key['g2']),
        "c" : cs.point_to_bytes(public_key['c']),
        "d" : cs.point_to_bytes(public_key['d']),
        "h" : cs.point_to_bytes(public_key['h'])
        }).transact({'from': w3.eth.accounts[0]})

tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [6]:
tx_hash = CramerShoup.functions.setChiperText(
    _ct = {
        "u1" : cs.point_to_bytes(chiper_text[0]),
        "u2" : cs.point_to_bytes(chiper_text[1]),
        "e" : cs.point_to_bytes(chiper_text[2]),
        "v" : cs.point_to_bytes(chiper_text[3])
        }).transact({'from': w3.eth.accounts[0]})

tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [7]:
tx_hash = CramerShoup.functions.encrypt(
    message = cs.point_to_bytes(message),
    k = "0x" + chiper_text[-1].to_bytes(32, 'big').hex()
    ).transact({'from': w3.eth.accounts[0]})
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [8]:
logs = CramerShoup.events.ChiperTextMathced().processReceipt(tx_receipt)
logs

(AttributeDict({'args': AttributeDict({'u1': [b'\x1ag\xcd\xd8\xb5Og\xca\xa2G\x98\xaf\xc6q\x18A\xc8G\xc9\xcdd\x00g>&%MR\xd3\xe5\xe4\x08',
    b'%\x1f2oxD%%\x99)\x9f\x15\x82T*\xe3M\xae\xa9\xd4\x83\xa1#\xa4\x812\xc2\xb0z\t\xe08'],
   'u2': [b'\x02\x83\x90\xb8\x9c\xaa\xab\xd5+.\xd2=\x85k\x81y\xf7r\xcabl)jk\xcd\x0e\xed\xf83{\x17\xb0',
    b'\x05m\xa0\xb1w\x9d\xbc\x0bS\xd3FEP\x8c\xf2\xec\x04\xd5cX\x8c\xf7c-\x99B2l\xbdcW\x8f'],
   'e': [b'$o\x14^\xd8\xe4f\x1e{\x0eA&\x95zo\xc4\xa7"\x17\xc3\xff\x8a\x1f\x19\x96Qq\x99\xf4\x0eG\xc5',
    b'\x01=\xc9\xf2wS\x93\x96\xfb\x95\xc6A\xd0\xb1,w\xc7,20\x91z\xa5\x14\x8c\x81\xa0+\xda|\xda\xa9'],
   'v': [b'\x15\xb9Y \x89\x0b\xa6\xaa\x0c\xacTC\x10\x17u\xca\x8dl4\xca\xb3\xde(F\x88$G\xdf\xfc4r\xab',
    b'\x1a\x9f\xa6\x8e\x01\xde\xe9\xaafIc\xfac\x9f\xa6\x00\x90\x82\xb5\x83y\x14\xf9\xec\xd7hI\x9cl5\xbe,']}),
  'event': 'ChiperTextMathced',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x7d86be1b27e6f13e76b72853a57f3c391a910e2ee7477477f5

In [9]:
try:
    tx_hash = CramerShoup.functions.encrypt(
        message = cs.point_to_bytes(message),
        k = "0x" + secrets.randbelow(bn128.field_modulus).to_bytes(32, 'big').hex()
        ).transact({'from': w3.eth.accounts[0]})
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
except Exception as e:
    print(e)

execution reverted: VM Exception while processing transaction: revert u1 not matched


In [5]:
dir(bn128)

['FQ',
 'FQ12',
 'FQ2',
 'FQP',
 'G1',
 'G12',
 'G2',
 'Z1',
 'Z2',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'absolute_import',
 'add',
 'b',
 'b12',
 'b2',
 'bn128_curve',
 'bn128_pairing',
 'curve_order',
 'double',
 'eq',
 'field_modulus',
 'final_exponentiate',
 'is_inf',
 'is_on_curve',
 'multiply',
 'neg',
 'pairing',
 'twist']

In [7]:
bn128.field_modulus % 4

3

In [28]:
(bn128.field_modulus + 1) // 4

5472060717959818805561601436314318772174077789324455915672259473661306552146

In [9]:
bn128.curve_order

21888242871839275222246405745257275088548364400416034343698204186575808495617

In [11]:
(-1024).to_bytes(32, byteorder='big')

OverflowError: can't convert negative int to unsigned

In [7]:
x = secrets.token_bytes(27)
x

b'\x01\x8e\xae)\xb95\xc5)\xb3\xc5P\x8a}\xb7A\x8e3_\x10^\xa1\x95(\xb48\x18\xf3'

In [10]:
ec_point = cs.encode_x_to_ec_point(x, 5)
ec_point

(640654580373707719519042747943061763503358300690071249367865587,
 16186217524233123527567967460095238571870155745959373512669242668003756854683)

In [11]:
cs.decode_ec_point_to_x(ec_point, 5)

b'\x01\x8e\xae)\xb95\xc5)\xb3\xc5P\x8a}\xb7A\x8e3_\x10^\xa1\x95(\xb48\x18\xf3'

In [12]:
Web3.solidityKeccak(['bytes'], [x])

HexBytes('0x4b370e0da91498dda09b3047b43226cc02f9ac1ebf296f7d293958e506667618')

In [25]:
import hashlib

# pad the message to 32 bytes
x = x + b'\x00' * (32 - len(x))

hashlib.sha256(x).digest().hex()

'6e8e418309f6d258f0db40013e03c90da6f51c2daed64e5b833ac1010102e8fd'

In [23]:
x.hex()


'018eae29b935c529b3c5508a7db7418e335f105ea19528b43818f3'